In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt

folder_path = 'C:/Users/dell/Desktop/project/stock_market_project/ticker_csvs' 

# Volitality Analysis
volatility_data = {}

# Loop through each CSV file in the folder
for filename in os.listdir(folder_path):
        ticker = filename.replace('.csv', '')
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)
    
        if 'date' in df.columns and 'close' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date')
            # calculate daily returns
            df['Previous Close'] = df['close'].shift(1)
            df['Daily Return'] = (df['close'] - df['Previous Close']) / df['Previous Close']

            # Calculate volatility (standard deviation of daily returns)
            volatility = df['Daily Return'].std()
            volatility_data[ticker] = volatility
            

volatility_df = pd.DataFrame(list(volatility_data.items()), columns=['Ticker', 'Volatility'])
volatility_df.to_csv('stock_volatility.csv', index=False)

In [97]:
# Cumulative returns

returns_data = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        ticker = filename.replace('.csv', '')
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)
 
        if 'date' in df.columns and 'close' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date')

            # Get start and end rows based on min and max dates
            start_row = df[df['date'] == df['date'].min()]
            end_row = df[df['date'] == df['date'].max()]

            if not start_row.empty and not end_row.empty:
                start_price = start_row.iloc[0]['close']
                end_price = end_row.iloc[0]['close']

                # Calculate cumulative return
                cumulative_return = (end_price - start_price) / start_price
                returns_data[ticker] = cumulative_return

returns_df = pd.DataFrame(list(returns_data.items()), columns=['Ticker', 'Cumulative Return'])
returns_df.to_csv('cumulative_returns.csv', index=False)


In [115]:
# Sector wise performance
sector_df = pd.read_csv('C:/Users/dell/Desktop/project/stock_market_project/sector_data.csv')

returns_df.columns = returns_df.columns.str.strip().str.lower()
merged_df = pd.merge(returns_df, sector_df, on='ticker')

# Group by sector and calculate average cumulative return
sector_avg = merged_df.groupby('sector')['cumulative return'].mean().reset_index()

# Sort descending by return
sector_avg = sector_avg.sort_values(by='cumulative return', ascending=False)
sector_avg.rename(columns={'cumulative return': 'average return'}, inplace=True)
sector_avg.to_csv('sector_performance.csv', index=False)

In [131]:
# Stock price correlation

returns_dict = {}

# Build daily return DataFrame
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        ticker = filename.replace('.csv', '')
        filepath = os.path.join(folder_path, filename)

        df = pd.read_csv(filepath)

        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date')

        df['prev_close'] = df['close'].shift(1)
        df['daily_return'] = (df['close'] - df['prev_close']) / df['prev_close']
        df.dropna(inplace=True)

        df.set_index('date', inplace=True)
        returns_dict[ticker] = df['daily_return']

returns_df = pd.DataFrame(returns_dict)
returns_long_df = returns_df.stack().reset_index()
returns_long_df.columns = ['date', 'ticker', 'daily return']
returns_long_df['date'] = pd.to_datetime(returns_long_df['date']).dt.date
returns_long_df.to_csv('daily_returns.csv', index=False)

correlation_matrix = returns_df.corr()
correlation_matrix.to_csv('stock_correlation_matrix.csv')


In [61]:
# Monthly Returns

monthly_returns = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        ticker = filename.replace('.csv', '')
        filepath = os.path.join(folder_path, filename)

        df = pd.read_csv(filepath)

        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date')
        df['month'] = df['date'].dt.to_period('M')

        # Group by month
        grouped = df.groupby('month')
        for period, group in grouped:
            first_close = group.iloc[0]['close']
            last_close = group.iloc[-1]['close']
            monthly_return = round(((last_close - first_close) / first_close) * 100, 2)
            monthly_returns.append({
                    'month': period.to_timestamp(),
                    'ticker': ticker,
                    'monthly_return': monthly_return
                }) 
            monthly_df = pd.DataFrame(monthly_returns)
            monthly_df.to_csv('monthly_returns.csv', index=False)

In [63]:
top_gainers = []
top_losers = []

for period in monthly_df['month'].unique():
    month_data = monthly_df[monthly_df['month'] == period]
    top5 = month_data.sort_values('monthly_return', ascending=False).head(5)
    bottom5 = month_data.sort_values('monthly_return').head(5)
    top_gainers.append(top5)
    top_losers.append(bottom5)

gainers_df = pd.concat(top_gainers)
losers_df = pd.concat(top_losers)
gainers_df.to_csv('top_monthly_gainers.csv', index=False)
losers_df.to_csv('top_monthly_losers.csv', index=False)

In [ ]:
!pip install pymysql

In [137]:
# Insert data to DB:
import pandas as pd
from sqlalchemy import create_engine

# MySQL connection settings
user = 'rino'
password = 'admin%40123'
host = 'localhost'        
port = 3306              
database = 'mydb'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

# 1. Insert into stock_volatility
volatility_df = pd.read_csv("stock_volatility.csv")
volatility_df.to_sql("stock_volatility", engine, if_exists="append", index=False)

# 2. Insert into cumulative_returns
cumulative_df = pd.read_csv("cumulative_returns.csv")
cumulative_df.rename(columns={'Cumulative Return': 'cumulative_return'}, inplace=True)
cumulative_df.to_sql("cumulative_returns", engine, if_exists="append", index=False)

# 3. Insert into sector_performance
sector_df = pd.read_csv("sector_performance.csv")
sector_df.rename(columns={'average return': 'average_return'}, inplace=True)
sector_df.to_sql("sector_performance", engine, if_exists="append", index=False)

# 4. Insert into daily_returns
daily_returns_df = pd.read_csv("daily_returns_rounded.csv")
daily_returns_df.columns = daily_returns_df.columns.str.lower()
daily_returns_df['date'] = pd.to_datetime(daily_returns_df['date'])
daily_returns_df.to_sql("daily_returns", engine, if_exists="append", index=False)

# 5. Insert into monthly_performance
monthly_df = pd.read_csv("monthly_returns.csv")
monthly_df.columns = monthly_df.columns.str.lower()
monthly_df['month'] = pd.to_datetime(monthly_df['month'])
monthly_df.to_sql("monthly_performance", engine, if_exists="append", index=False)

engine.dispose()